## Bad SQL Localization

In [1]:
import sys
import matplotlib.pyplot as plt
%matplotlib inline
sys.path.append('..')
import json
import pandas as pd
import numpy as np
import os
import time
import warnings
# ----------------infos----------------------
print("numpy version: {}".format(np.__version__))
# np.show_config()
print("pandas version: {}".format(pd.__version__))
from evaluate import evaluate_all
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

numpy version: 1.23.3
pandas version: 1.5.0


In [2]:
!pwd

/Users/chency/PythonProjects/balance/bad_sql_experiments


## 读取训练数据， original from OSS

In [4]:
csv_dir_path = "bad_sql_data/"
record = pd.read_csv(os.path.join(csv_dir_path, 'record.csv'))
sql_ = pd.read_csv(os.path.join(csv_dir_path, 'sql.csv'))
metric = pd.read_csv(os.path.join(csv_dir_path, 'metric.csv'))

record['occur_time'] = pd.to_datetime(record['occur_time'])
sql_['batch_time'] = pd.to_datetime(sql_['batch_time'])
metric['event_date'] = pd.to_datetime(metric['event_date'])

record_filtered = record.loc[(record.occur_time > '2021/6/2')]
record_filtered = record_filtered.sort_values('occur_time')
clusters = record_filtered.cluster
timestamps = record_filtered.occur_time
tenants = record_filtered.tenant_name
labels = record_filtered.sql_id

print(len(record_filtered), len(labels))

96 96


## evaluate BMFS

In [5]:
def evaluate_BMFS(metric, sql_, clusters, tenants, timestamps, labels, after=1):
    result_rows = []
    for impute in [False]:
        for positive in [True]:
            for normalize in ['allstd']:
                for attribution_method in ['gradients_x_inputs',]:
                    hit_num, miss_num, _, _, avg_rec_num, avg_time = evaluate_all(metric, sql_, clusters, tenants, timestamps, labels, after=after, trainer='bayesFS', attribution_method=attribution_method, impute=impute, positive=positive, normalize=normalize)
                    hit_rate = hit_num / (hit_num + miss_num)
                    row = {'impute': impute,
                          'positive': positive,
                          'normalize': normalize,
                          'attribution_method': attribution_method,
                          'Accuracy(top3)': hit_rate,
                           'avg_rec_num': avg_rec_num,
                           "avg_time": avg_time}
                    result_rows.append(row)
    return pd.DataFrame(result_rows)

In [ ]:
result_df = evaluate_BMFS(metric, sql_, clusters, tenants, timestamps, labels, after=1)

In [8]:
result_df

,impute,positive,normalize,attribution_method,Accuracy(top3),avg_rec_num,avg_time
0,False,True,allstd,gradients_x_inputs,0.833333,2.277778,1.121089


## evaluate fsMTS

In [6]:
def evaluate_fsMTS(metric, sql_, clusters, tenants, timestamps, labels, after=1):
    result_rows = []
    for impute in [True]:
        for positive in [True]:
            for normalize in ['allstd']:
                for attribution_method in ['gradients']:
                    hit_num, miss_num, _, _, avg_rec_num, avg_time = evaluate_all(metric, sql_, clusters, tenants, timestamps, labels, after=after, trainer='fsMTS', attribution_method=attribution_method, impute=impute, positive=positive, normalize=normalize, leftshit=False)
                    hit_rate = hit_num / (hit_num + miss_num)
                    row = {'impute': impute,
                          'positive': positive,
                          'normalize': normalize,
                          'attribution_method': attribution_method,
                          'Accuracy(top3)': hit_rate,
                           'avg_rec_num': avg_rec_num,
                           "avg_time": avg_time}
                    result_rows.append(row)
    return pd.DataFrame(result_rows)

In [ ]:
result_df = evaluate_fsMTS(metric, sql_, clusters, tenants, timestamps, labels, after=1)

In [ ]:
result_df

## evaluate enet

In [3]:
def evaluate_enet(metric, sql_, clusters, tenants, timestamps, labels, after=1):
    result_rows = []
    for impute in [True]:
        for positive in [True]:
            for normalize in ['allstd']:
                for attribution_method in ['gradients_x_inputs']:
                    hit_num, miss_num, _, _, avg_rec_num, avg_time = evaluate_all(metric, sql_, clusters, tenants, timestamps, labels, after=after, trainer='myenet', attribution_method=attribution_method, impute=impute, positive=positive, normalize=normalize)
                    hit_rate = hit_num / (hit_num + miss_num)
                    row = {'impute': impute,
                          'positive': positive,
                          'normalize': normalize,
                          'attribution_method': attribution_method,
                          'Accuracy(top3)': hit_rate,
                           'avg_rec_num': avg_rec_num,
                           "avg_time": avg_time}
                    result_rows.append(row)
    return pd.DataFrame(result_rows)

In [ ]:
result_df = evaluate_enet(metric, sql_, clusters, tenants, timestamps, labels, after=1)

In [6]:
result_df

## evaluate ARD

In [7]:
def evaluate_ard(metric, sql_, clusters, tenants, timestamps, labels, after=1):
    result_rows = []
    for impute in [False]:
        for positive in [True]:
            for normalize in ['allstd']:
                for attribution_method in ['gradients_x_inputs']:
                    hit_num, miss_num, _, _, avg_rec_num, avg_time = evaluate_all(metric, sql_, clusters, tenants, timestamps, labels, after=after, trainer='myard', attribution_method=attribution_method, impute=impute, positive=positive, normalize=normalize)
                    hit_rate = hit_num / (hit_num + miss_num)
                    row = {'impute': impute,
                          'positive': positive,
                          'normalize': normalize,
                          'attribution_method': attribution_method,
                          'Accuracy(top3)': hit_rate,
                           'avg_rec_num': avg_rec_num,
                           "avg_time": avg_time}
                    result_rows.append(row)
    return pd.DataFrame(result_rows)

In [ ]:
result_df = evaluate_ard(metric, sql_, clusters, tenants, timestamps, labels, after=1)
result_df

## evaluate LASSO

In [7]:
def evaluate_lasso(metric, sql_, clusters, tenants, timestamps, labels, after=1):
    result_rows = []
    for impute in [True]:
        for positive in [True]:
            for normalize in ['allstd']:
                for attribution_method in ['gradients_x_inputs']:
                    hit_num, miss_num, _, _, avg_rec_num, avg_time = evaluate_all(metric, sql_, clusters, tenants, timestamps, labels, after=after, trainer='mylasso', attribution_method=attribution_method, impute=impute, positive=positive, normalize=normalize)
                    hit_rate = hit_num / (hit_num + miss_num)
                    row = {'impute': impute,
                          'positive': positive,
                          'normalize': normalize,
                          'attribution_method': attribution_method,
                          'Accuracy(top3)': hit_rate,
                           'avg_rec_num': avg_rec_num,
                           "avg_time": avg_time}
                    result_rows.append(row)
    return pd.DataFrame(result_rows)

In [ ]:
result_df = evaluate_lasso(metric, sql_, clusters, tenants, timestamps, labels, after=1)

In [ ]:
result_df